## Data Cleaning
- Outliers
    - Outlier removal based on z-score. The z-score is calculated on the user input variable and values with absolute z-score greater than threshold_z for the user input granularity.
- Standardization
    - Standardization of SKU/Store names and finding top three matches based on fuzzy score.
    - User inputs two lists of strings. First list will be reference list and top three matches will be found from second list.
    - Takes 16 min on model dataset with 800 x 800 combinations.
- Output file location: Output/Data_Cleaning

In [ ]:
import numpy as np
import pandas as pd
import datetime
import time
pd.options.mode.chained_assignment = None
import re
import jellyfish as jf
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from joblib import Parallel, delayed

In [ ]:
#!pip3 install fuzzywuzzy

- Outliers function

In [ ]:
def outliers(df,
             variable, # column on which to find z-score e.g Sales column
             year_col,
             period_level,
             gran_list, # list of granularity e.g. ['Segment','Brand','APN']
             threshold_z = 1.96): 
    
    """
    Function to remove outliers using z-score from the dataset. The user can input the variable on which to perform z-score.
    The values with absolute z-score >= 1.96 are removed.
    
        Parameters in order:
            df: Dataframe
            variable: The column on which to perform z-score e.g. Unit sales column
            year_col: Year column
            period_level: The aggregating column used for calculation of mean and standard deviation for the input 'variable'.
                          Can be Month column, Quarter column, etc.
            gran_list (list): List of granularity column across which calculation is to be done. e.g. ['Segment','Brand','SKU']
                                for Segment x Brand x SKU
            threshold_z (float): Threshold z-score value. Default= 1.96
        Returns:
            The original dataframe with outliers removed
    """
    gran_list.extend([year_col,period_level])
    df['Std'] = df.groupby(gran_list)[variable].transform('std') # finding standard deviation
    df['Mean'] = df.groupby(gran_list)[variable].transform('mean') # finding mean
    
    # calculating z-score for entries
    df['z-score'] = abs(df[variable]-df['Mean'])/df['Std']
    
    # filtering data with z-score > threshold_z
    df = df[df['z-score'] < threshold_z]
    
    df = df.reset_index(drop=True)
    
    df.drop(columns=['Std','Mean','z-score'],axis=1,inplace=True)
    
    df.to_csv('../Output/Data_Cleaning/Outliers_removed.csv',index=False)
    
    return df

- Standardization function

In [ ]:
# Dependencies for main functions
# function to covert to utf-8 string
def to_unicode(obj, encoding='utf-8'):
    if not isinstance(obj, str):
        return obj.decode(encoding, errors='ignore')
    return obj


def f_get_modified_str(input_str):
    # Transform to lower case
    input_str = input_str.strip().lower()
    # Replace '&' char to word 'and'
    input_str = input_str.replace(' & ', ' and ')
    input_str = input_str.replace('&', '')
    # Remove all other special chars excluding '.'
    input_str = " ".join(re.findall("[a-zA-Z0-9.]+", input_str))
    # Remove extra spaces
    input_str = " ".join(input_str.split()) # split saves words of string as list
    return input_str.strip()

#name1(Tesco) = Mars
#name2 (Efun)= Mars
#score 100

#name1 = Mars
#name2 = Galaxy
#score 5

def name_match(name1, name2):
    ratio1=0.0
    ratio2=0.0
    name1 = name1.split()
    name2 = name2.split()
    if name1 and name2:
        ratio1 = 0.0
        for i in range(len(name1)):
            for j in range(i, len(name2)):
                if fuzz.ratio(name1[i], name2[j]) > 90:
                    ratio1 += 1.0
                    break
        try:
            ratio1 = 100 * (ratio1 / len(name1))
        except ZeroDivisionError:
            ratio1 = 0.0
        ratio2 = 0.0
        for i in range(len(name2)):
            for j in range(i, len(name1)):
                if fuzz.ratio(name2[i], name1[j]) > 90:
                    ratio2 += 1.0
                    break
        try:
            ratio2 = 100 * (ratio2 / len(name2))
        except ZeroDivisionError:
            ratio2 = 0.0
    return int(max(ratio1, ratio2))


# Function to calculate fuzzy match score for two given strings

def f_name_match_score(str1, str2):
    """
    Main function to modify the strings and calculate fuzzy scores.
    
    Returns score of two strings.
    """
    # Convert to unicode to avoid error
    
    str1 = to_unicode(str1)
    str2 = to_unicode(str2)
    # Transform name
    str1 = f_get_modified_str(str1)
    str2 = f_get_modified_str(str2)
    # Compute match scores
    score1 = fuzz.ratio(str1, str2)
    score2 = fuzz.token_sort_ratio(str1, str2)
    score3 = jf.jaro_winkler(str1, str2) * 100
    score4 = fuzz.WRatio(str1,str2)
    score6 = name_match(str1,str2)
    score5 = 0
    if ((len(str2.split()) >= 1) & (len(str1.split()) >= 1)):
        score5 = fuzz.ratio(str1.split()[0], str2.split()[0])
    if (score5>0):
        s_max = (score1 + score2 +score3 + score4 + score5 + score6)/6
    else:
        s_max = (score1 + score2 +score3 + score4 + score6)/5

    return s_max

# Main function
def sku_standardize(str_list_1, str_list_2):
    """
    Function to output string list and its top 3 matches for each string from another string list based on 
    fuzzy score calculation.
    
        Parameters in order:
            str_list_1(list)(str) : String list 1
            str_list_2(list)(str) : String list from which to get best matches
            
        Returns:
            Dataframe with strings, their top 3 matches and score out of 100
            
    """
    index = pd.MultiIndex.from_product([str_list_1, str_list_2], names = ["Name", "Options"])
    index = pd.DataFrame(index = index).reset_index()
    scores = []
    for i in str_list_1:
        element_run = Parallel(n_jobs=4)(delayed(f_name_match_score)(i,j) for j in str_list_2)
        scores.extend(element_run)
    index['Score'] = scores
    df_agg = index.groupby(['Name','Options']).agg({'Score':sum})
    df_agg = df_agg['Score'].groupby('Name', group_keys=False).nlargest(3).reset_index()
    df_agg['Rank'] = list(np.arange(1,4))*len(str_list_1)
    
    df_agg.to_excel('../Output/Data_Cleaning/Standardization-Top_matches.xlsx',index=False)
    
    return df_agg


### Validation on Model dataset

- Outliers

In [ ]:
df = pd.read_excel('../Data/Testing_Data.xlsx')

In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df.head()

,Segment,Date,Retailer,Manufacturer,APN,Packsize,Brand,Sales,Unit_sales,Vol_Sales,wtd_distribution,PPG_Item_No,Year,Month
0,Bitesize,2017-04-02,Coles,Mars Wrigley,5000159491532,88g,Galaxy,9201.320,2570.0,226.160,75.9,Item_5000159491532,2017,4
1,Bitesize,2017-04-02,Coles,Mars Wrigley,5000159494762,149g,M&M's,61.760,61.0,9.089,1.9,Item_5000159494762,2017,4
2,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682002134,140g,Maltesers,178508.416,61625.0,8627.500,99.9,Item_9300682002134,2017,4
3,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682013376,145g,M&M's,87435.360,30179.0,4375.955,100.0,Item_9300682013376,2017,4
4,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682015479,145g,M&M's,30701.950,10589.0,1535.405,95.9,Item_9300682015479,2017,4


In [ ]:
ds = outliers(df,'Unit_sales','Year','Month',['Segment','Retailer','Manufacturer','Brand'])

In [ ]:
ds

,Segment,Date,Retailer,Manufacturer,APN,Packsize,Brand,Sales,Unit_sales,Vol_Sales,wtd_distribution,PPG_Item_No,Year,Month
0,Bitesize,2017-04-02,Coles,Mars Wrigley,5000159491532,88g,Galaxy,9201.320,2570.0,226.160,75.9,Item_5000159491532,2017,4
1,Bitesize,2017-04-02,Coles,Mars Wrigley,5000159494762,149g,M&M's,61.760,61.0,9.089,1.9,Item_5000159494762,2017,4
2,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682015479,145g,M&M's,30701.950,10589.0,1535.405,95.9,Item_9300682015479,2017,4
3,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682026079,145g,M&M's,39682.610,13699.0,1986.355,95.8,Item_9300682026079,2017,4
4,Bitesize,2017-04-02,Coles,Mars Wrigley,9300682030892,180g,Pods,483.550,160.0,28.800,14.6,Item_9300682030892,2017,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91676,Block,2020-03-22,Woolworths,All Other Manufacturer,9403142002375,200g,Whittakers,4.796,1.0,0.200,0.2,Item_9403142002375,2020,3
91677,Block,2020-03-22,Woolworths,All Other Manufacturer,9403142004652,100g,Whittakers,2186.008,489.0,48.900,26.9,Item_9403142004652,2020,3
91678,Block,2020-03-22,Woolworths,All Other Manufacturer,9403142004836,100g,Whittakers,2678.324,600.0,60.000,27.8,Item_9403142004836,2020,3
91679,Block,2020-03-22,Woolworths,Lindt & Sprungli,9542009984,100g,Lindt Excellence,39306.531,11410.0,1141.000,100.0,Item_9542009984,2020,3


- Standardization

In [ ]:
df2 = pd.read_excel('../Data/SKU-standardisation.xlsx','Sheet2')

In [ ]:
list1 = list(df2['A'])
list2 = list(df2['B'])
list1 = [x for x in list1 if str(x) != 'nan']
list2 = [x for x in list2 if str(x) != 'nan']

In [ ]:
df_ag = sku_standardize(list1,list2)

In [ ]:
df3 = pd.read_excel('../Data/SKU-standardisation.xlsx','Sheet1')

In [ ]:
list1 = list(df3['Product Name'])
list2 = list(df3['Description_english'])
list1 = [x for x in list1 if str(x) != 'nan']
list2 = [x for x in list2 if str(x) != 'nan']

In [ ]:
s = time.time()
df_ag2 = sku_standardize(list1,list2)
e = time.time()

In [ ]:
print(e-s)

1044.752935886383


In [ ]:
df3 = pd.read_excel('../Data/SKU-standardisation.xlsx','Sheet3')
list1 = list(df3['Activity Name'])
list2 = list(df3['Short Text'])
list1 = [x for x in list1 if str(x) != 'nan']
list2 = [x for x in list2 if str(x) != 'nan']

In [ ]:
list1

['Spring Multibrand',
 'Easter',
 'Back-2-School Multibrand',
 'New Year Multibrand',
 'Snickers EBA + LE Cream',
 'Big Bar promo',
 'Snickers Crisper',
 'Twix White',
 'Twix Hot Beverages EBA',
 'Snickers Stick',
 'Snickers Hunger Bar EBA',
 'Twix Salted Caramel',
 'Snickers White LE',
 'Orbit Dating',
 'Orbit Car Occasion',
 'Orbit Multibrand',
 'Orbit Mega',
 'Orbit Car Occasion 2.0',
 'Orbit Dating 2.0',
 "Korkunov Teacher's Day",
 'Dacha Multibrand',
 'Five EBA 5 Adventures',
 'Five AGM',
 'Five Gaming EBA',
 'M&Ms You&Me',
 'Skittles Spider Man',
 'M&Ms Tablet',
 'M&Ms Screentime 1.0',
 'Bitesize Screentime 2.0']

In [ ]:
list2

['Orbit D-Com Plan Digital',
 'Easter 2020 national KV + recipes',
 "RU_M&M's_Screentime EBA_0420-Plan-",
 '6470200,00 - Buzova',
 'BE-KIND Rosa Ski resort activation 2',
 'Orbit White Mega campaign',
 "16_CE_M&M's Customization for retail (Ma",
 'Premia_Go-pro_SNK EBA P3-4 2020',
 'Premia_JBL_SNK EBA P3-4 2020',
 'Premia_Camera_SNK EBA P3-4 2020',
 'Premia_Playstation_SNK EBA P3-4 2020',
 'Premia_Iwatch_SNK EBA P3-4 2020',
 "RU_M&M's_Screentime EBA Social Support_0",
 'Five - KV for Soft Chew',
 'Skittles Ex-Football 2020_TMA Execution',
 'Five_Shmot_LoL Licence',
 'MMs_ivi_Content Rolls campaign_20200211',
 'Dove Caramel NY LE',
 'BE-KIND nutrition design',
 'Five Gaming fonts purchase',
 '27_CE_MMs_Statis banner for Okey',
 'BE-KIND act of kindess Perito',
 'BE-KIND METRO Expo 2020',
 '361.604,80 - MEGA (OOH Universities)',
 'Five gaming streamers',
 'SNK & Orbit KV development',
 '05_SNK NY Tube Design Development',
 'TMA Presentation design',
 'SNK EBA 2020 banner production',
 'F

In [ ]:
df_ag3 = sku_standardize(list1,list2)

In [ ]:
df_ag3

,Name,Options,Score,Rank
0,Back-2-School Multibrand,Spring 2020 multibrand,55.369408,1
1,Back-2-School Multibrand,NY 19-20 Tander Multibrand mechanics,53.144444,2
2,Back-2-School Multibrand,NY 19-20 Multibrand in Tander (KKV part),50.553801,3
3,Big Bar promo,SNK big bar deisgn // Celebrity gift,56.137270,1
4,Big Bar promo,Orbit Dating OLV promo tag,55.636752,2
...,...,...,...,...
82,Twix Salted Caramel,TWIX salted caramel digital plan,88.645833,2
83,Twix Salted Caramel,Twix Salted Caramel TVC Adaptation,87.529412,3
84,Twix White,Twix White & Top KV for internal use,76.543860,1
85,Twix White,TWIX Website update,74.208577,2
